In [2]:
# Core
import numpy as np
import pandas as pd

import time

In [3]:
# Local imports
import kepler_sieve
from asteroid_skypatch import calc_ast_skypatch
from asteroid_skypatch_test import test_skypatch_dir
from asteroid_direction import c, mpd, dpm, prep_ast_block, calc_dir_ast_block
from asteroid_spline import make_spline_ast_dir
from sky_patch import dir2SkyPatchID, SkyPatchID2dir
from db_utils import sp2df, df2db

# Build block of asteroid directions at high resolution

In [7]:
# Asteroid block
n0: int = 1
n1: int = 100

# Time range for asteroid directions
mjd0: int = 58000
mjd1: int = 60000
interval_min: int = 15

In [8]:
# # Asteroid block
# n0: int = 1
# n1: int = 3

# # Time range for asteroid directions
# mjd0: int = 58000
# mjd1: int = 60000
# interval_min: int = 1440

In [9]:
t0 = time.time()
df = calc_ast_skypatch(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1, interval_min=interval_min)
t1 = time.time()

In [10]:
et = t1 - t0
print(f'Elapsed time with interval={interval_min}: {et:5.2f}')

Elapsed time with interval=15: 34.78


In [11]:
df

,AsteroidID,Segment,SkyPatchID,TimeID_0,TimeID_1
0,1,0,6382172,83520000,83520135
1,1,1,6382171,83520135,83520225
2,1,2,6384219,83520225,83520315
3,1,3,6384218,83520315,83520495
4,1,4,6384217,83520495,83520675
...,...,...,...,...,...
1413041,99,14094,6606593,86398875,86399130
1413042,99,14095,6606592,86399130,86399385
1413043,99,14096,6606591,86399385,86399640
1413044,99,14097,6606590,86399640,86399895


In [12]:
# Arguments to df2db
schema = 'KS'
table = f'AsteroidSkyPatch_Stage'
columns = ['AsteroidID', 'Segment', 'SkyPatchID', 'TimeID_0', 'TimeID_1']
chunksize = 2**19
verbose = True
progbar = True

# Dispatch to df2db
df2db(df=df, schema=schema, table=table, columns=columns, chunksize=chunksize, verbose=verbose, progbar=progbar)

Extracting 1413046 records from DataFrame into CSV files in chunks of 524288 rows...
CSV file name: /home/michael/Harvard/kepler-sieve/data/df2db/AsteroidSkyPatch_Stage/pid_1837358/AsteroidSkyPatch_Stage.csv
Elapsed Time for CSV Conversion: 0:00:06.101873


  0%|          | 0/3 [00:00<?, ?it/s]

Elapsed Time for DB insertion: 0:12:12.360831


# Test direction at midpoint of interval vs. center of sky patch

In [8]:
# Time at the midpoint of each interval
t_obs_0 = df.TimeID_0.values / mpd
t_obs_1 = df.TimeID_1.values / mpd
t_obs = (t_obs_0 + t_obs_1)/2.0

In [9]:
df['tObs'] = t_obs

In [10]:
# Extract arrays
asteroid_id = df.AsteroidID.values
sky_patch_id = df.SkyPatchID.values

In [11]:
# # Direction used to build sky patch DataFrame
# cols_u = ['ux', 'uy', 'uz']
# u_df = df[cols_u].values

In [12]:
# Spline for asteroid directions vs. time
spline_u = make_spline_ast_dir(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1)

In [13]:
# Evaluate splined asteroid directions
u_ast, lt_ast = spline_u(ts=t_obs, asteroid_id=asteroid_id)

In [14]:
# Direction at the center of each sky patch
N_sp = 1024
u_sky = SkyPatchID2dir(SkyPatchID=sky_patch_id, N=N_sp)

In [15]:
# Compare splined asteroid direction to center of skypatch
du = test_skypatch_dir(name1='spline', name2='skypatch', u1=u_ast, u2=u_sky, verbose=True)

Angle Difference: skypatch vs. spline in arc seconds
*Mean  :      48.9*
 Median:      45.9
 Max   :     144.4


In [19]:
from asteroid_element import get_asteroids
ast = get_asteroids()

In [20]:
ast

,AsteroidID,BodyID,AsteroidName,BodyName
AsteroidID,,,,
1,1,1000001,Ceres,SB.Ceres
2,2,1000002,Pallas,SB.Pallas
3,3,1000003,Juno,SB.Juno
4,4,1000004,Vesta,SB.Vesta
5,5,1000005,Astraea,SB.Astraea
...,...,...,...,...
1413590,1413590,2413590,6013 P-L,SB.6013 P-L
1413591,1413591,2413591,6331 P-L,SB.6331 P-L
1413592,1413592,2413592,6344 P-L,SB.6344 P-L


In [21]:
sz = 100000
jn = 3

In [22]:
i0 = sz*jn
i1 = i0+sz

In [24]:
asteroid_id = ast.AsteroidID.values
n0 = asteroid_id[i0]
n1 = asteroid_id[i1]

300001

In [23]:
from detection_enrich import calc_detections
from sky_patch import N_sp
from db_utils import sql_run
from tqdm.auto import tqdm as tqdm_auto

In [14]:
n = 2
sz = 100000
did0 = n*sz
did1 = did0 + sz - 1
params = {
    'did0': did0,
    'did1': did1,
}

In [19]:
sql_str = \
"""
INSERT INTO KS.Detection_temp
(DetectionID, DetectionTimeID, SkyPatchID, TimeID, mjd, ux, uy, uz, mag)
SELECT
	det.DetectionID,
	det.DetectionTimeID,
	det.SkyPatchID,
	CAST(FLOOR(det.mjd*1440) AS INT) AS TimeID,
	det.mjd,
	det.ux,
	det.uy,
	det.uz,
	det.mag
FROM
	KS.Detection_v1 AS det
WHERE det.DetectionID BETWEEN :did0 AND :did1;
"""

In [20]:
# sql_run(sql_str=sql_str, params=params)

In [24]:
for n in tqdm_auto(range(3, 100)):
    pass

  0%|          | 0/97 [00:00<?, ?it/s]

In [25]:
# Batch size
sz = 100000
# Iterate over batches
for n in tqdm_auto(range(3, 100)):
    # Range of DetectionID
    did0 = n*sz
    did1 = did0 + sz - 1
    # Bind parameters and run SQL
    params = {
        'did0': did0,
        'did1': did1,
    }
    sql_run(sql_str=sql_str, params=params)

  0%|          | 0/97 [00:00<?, ?it/s]

sql_run failed!

INSERT INTO KS.Detection_temp
(DetectionID, DetectionTimeID, SkyPatchID, TimeID, mjd, ux, uy, uz, mag)
SELECT
	det.DetectionID,
	det.DetectionTimeID,
	det.SkyPatchID,
	CAST(FLOOR(det.mjd*1440) AS INT) AS TimeID,
	det.mjd,
	det.ux,
	det.uy,
	det.uz,
	det.mag
FROM
	KS.Detection_v1 AS det
WHERE det.DetectionID BETWEEN :did0 AND :did1;

Traceback (most recent call last):
  File "/home/michael/Harvard/kepler-sieve/src/db_utils.py", line 120, in sql_run
    conn.execute(sql_stmt)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/sqlalchemy/engine/base.py", line 1123, in execute
    return meth(self, multiparams, params, _EMPTY_EXECUTION_OPTS)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/sqlalchemy/sql/elements.py", line 310, in _execute_on_connection
    return connection._execute_clauseelement(
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/sqlalchemy

In [ ]:
for n in tqdm_auto(range(100, 1000)):
    # Range of DetectionID
    did0 = n*sz
    did1 = did0 + sz - 1
    # Bind parameters and run SQL
    params = {
        'did0': did0,
        'did1': did1,
    }
    sql_run(sql_str=sql_str, params=params)